In [1]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pandas

import os
import pandas as pd

# ==========================
# CONFIGURAÇÕES INICIAIS
# ==========================

# Diretório onde estão os arquivos de resultado
resultados_dir = r'G:\Meu Drive\IPEA\Consulta_autom'

# Lista de anos a serem processados
anos = [2019, 2020, 2021, 2022, 2023, 2024]

# Nome do arquivo final consolidado
arquivo_final = 'consolidado_politicas.csv'

# Criar um DataFrame vazio para armazenar os resultados finais
df_final = pd.DataFrame()

# ==========================
# LEITURA E CONSOLIDAÇÃO POR ANO
# ==========================

for ano in anos:
    arquivo_csv = os.path.join(resultados_dir, f"resultado_{ano}.csv")

    # Verifica se o arquivo existe antes de tentar ler
    if not os.path.exists(arquivo_csv):
        print(f"❌ Arquivo não encontrado: {arquivo_csv}, pulando para o próximo ano...")
        continue

    print(f"📂 Lendo {arquivo_csv}...")

    # Carrega o arquivo CSV
    df = pd.read_csv(arquivo_csv)

    # Ajuste para remover espaços extras nos nomes das colunas
    df.columns = df.columns.str.strip()

    # Consolida os dados agrupando por política e somando quantidades
    df_agrupado = df.groupby(['politica', 'ano_politica'], as_index=False).agg({
        f'qtde_{ano}': 'sum',  # Soma as quantidades
        f'paginas_{ano}': lambda x: ','.join(sorted(set(str(i) for i in x if pd.notna(i))))  # Junta as páginas únicas
    })

    # Junta os dados de cada ano no DataFrame final
    if df_final.empty:
        df_final = df_agrupado
    else:
        df_final = pd.merge(df_final, df_agrupado, on=['politica', 'ano_politica'], how='outer')

# ==========================
# EXPORTAÇÃO DO ARQUIVO FINAL
# ==========================

# Ordenar os resultados por ano_politica e nome da política
df_final = df_final.sort_values(by=['ano_politica', 'politica'])

# Salvar o resultado consolidado em CSV
df_final.to_csv(os.path.join(resultados_dir, arquivo_final), index=False, encoding='utf-8-sig')

print(f"\n✅ Consolidação concluída! Arquivo salvo em: {os.path.join(resultados_dir, arquivo_final)}")


📂 Lendo G:\Meu Drive\IPEA\Consulta_autom\resultado_2019.csv...
📂 Lendo G:\Meu Drive\IPEA\Consulta_autom\resultado_2020.csv...
📂 Lendo G:\Meu Drive\IPEA\Consulta_autom\resultado_2021.csv...
📂 Lendo G:\Meu Drive\IPEA\Consulta_autom\resultado_2022.csv...
📂 Lendo G:\Meu Drive\IPEA\Consulta_autom\resultado_2023.csv...
📂 Lendo G:\Meu Drive\IPEA\Consulta_autom\resultado_2024.csv...

✅ Consolidação concluída! Arquivo salvo em: G:\Meu Drive\IPEA\Consulta_autom\consolidado_politicas.csv


In [12]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pandas chardet openpyxl

import os
import pandas as pd
import chardet

# ==========================
# CONFIGURAÇÕES INICIAIS
# ==========================

# Diretório onde estão os arquivos
dados_dir = r'G:\Meu Drive\IPEA\Consulta_autom'

# Arquivos de entrada
arquivo_consolidado = os.path.join(dados_dir, "consolidado_politicas.csv")
arquivo_base = os.path.join(dados_dir, "politicas-base-organizada.csv")

# Nome do arquivo final atualizado
arquivo_final = os.path.join(dados_dir, "consolidado_politicas_atualizado.csv")

# ==========================
# FUNÇÃO PARA DETECTAR ENCODING E DELIMITADOR
# ==========================
def detectar_encoding_e_delimitador(arquivo):
    """ Detecta a codificação e o delimitador de um arquivo CSV. """
    with open(arquivo, "rb") as f:
        encoding_detectado = chardet.detect(f.read(100000))["encoding"]

    # Tentar identificar o delimitador correto
    with open(arquivo, "r", encoding=encoding_detectado, errors="replace") as f:
        primeira_linha = f.readline()
        delimitador = "," if "," in primeira_linha else ";" if ";" in primeira_linha else "\t"

    return encoding_detectado, delimitador

# ==========================
# LEITURA DOS ARQUIVOS COM DETECÇÃO AUTOMÁTICA
# ==========================

print("📂 Detectando encoding e delimitador dos arquivos...")

# Detectar encoding e delimitador do consolidado_politicas.csv
encoding_consolidado, sep_consolidado = detectar_encoding_e_delimitador(arquivo_consolidado)
df_consolidado = pd.read_csv(arquivo_consolidado, encoding=encoding_consolidado, sep=sep_consolidado, on_bad_lines="skip", low_memory=False)

# Detectar encoding e delimitador do politicas-base-organizada.csv
encoding_base, sep_base = detectar_encoding_e_delimitador(arquivo_base)
df_base = pd.read_csv(arquivo_base, encoding=encoding_base, sep=sep_base, on_bad_lines="skip", low_memory=False)

print(f"✅ Arquivos lidos com sucesso!\n  📂 Consolidado: {encoding_consolidado}, delimitador: {sep_consolidado}\n  📂 Base: {encoding_base}, delimitador: {sep_base}")

# ==========================
# PADRONIZAÇÃO DOS DADOS
# ==========================

# Remover espaços extras dos nomes das colunas e dados das políticas
df_consolidado.columns = df_consolidado.columns.str.strip()
df_base.columns = df_base.columns.str.strip()

df_consolidado["politica"] = df_consolidado["politica"].str.strip()
df_base["nome"] = df_base["nome"].str.strip()

# Seleciona apenas as colunas desejadas da base qualitativa
colunas_desejadas = [
    "nome", "cat_nome", "grande_area", "area", "orgao_nome",
    "instrumento_legal", "legislacao", "vigencia_inicio", "vigencia_fim"
]

df_base = df_base[colunas_desejadas]

# Renomear "nome" para "politica" para facilitar a mesclagem
df_base = df_base.rename(columns={"nome": "politica"})

# Remover duplicatas, mantendo apenas o primeiro registro encontrado para cada política
df_base = df_base.drop_duplicates(subset=["politica"], keep="first")

# ==========================
# INCREMENTAR O ARQUIVO CONSOLIDADO
# ==========================

print("🔄 Mesclando dados...")

# Faz o merge para adicionar as informações da base qualitativa ao consolidado
df_final = df_consolidado.merge(df_base, on="politica", how="left")

# ==========================
# CORREÇÃO DO FORMATO DA COLUNA 'ANO_POLITICA'
# ==========================

df_final["ano_politica"] = pd.to_numeric(df_final["ano_politica"], errors="coerce").fillna(0).astype("Int64")

print("✅ Consolidação final realizada com sucesso!")

# ==========================
# EXPORTAÇÃO DO ARQUIVO FINAL
# ==========================

# Ordenar os resultados por ano_politica e nome da política
df_final = df_final.sort_values(by=['ano_politica', 'politica'])

# Salvar o arquivo final atualizado
df_final.to_csv(arquivo_final, index=False, encoding='utf-8-sig')

print(f"\n✅ Atualização concluída! Arquivo salvo em: {arquivo_final}")


📂 Detectando encoding e delimitador dos arquivos...
✅ Arquivos lidos com sucesso!
  📂 Consolidado: Windows-1252, delimitador: 	
  📂 Base: utf-8, delimitador: ,
🔄 Mesclando dados...
✅ Consolidação final realizada com sucesso!

✅ Atualização concluída! Arquivo salvo em: G:\Meu Drive\IPEA\Consulta_autom\consolidado_politicas_atualizado.csv
